In [1]:
import copy
import fnmatch
import json
import getpass
import os
import pathlib
import datetime
                    
from dask.distributed import LocalCluster, SSHCluster 
from laserfarm import Retiler, DataProcessing, GeotiffWriter, MacroPipeline
from laserfarm.remote_utils import get_wdclient, get_info_remote, list_remote

def last_modified(opts, remote_path):
    info = get_info_remote(get_wdclient(opts), remote_path.as_posix())
    format_ = '%a, %d %b %Y %H:%M:%S GMT'
    return datetime.datetime.strptime(info['modified'], format_)

# Macro-Pipeline Workflow - GeoTIFF Export (Mask)

## Set Run-Specific Input

Fill in the username/password for the SURF dCache. 

In [2]:
path_root = pathlib.Path('/project/lidarac/Data')

# We have mounted the dCache system to read input
path_input = path_root / 'AHN4_mask/TOP10NL_2021_shapefiles/targets_all'

# dCache path where to copy the geotiff files
path_output = path_root / 'AHN4_mask/TOP10NL_2021_shapefiles/geotiff_classified'

## Setup Cluster

Setup Dask cluster used for all the macro-pipeline calculations.

In [ ]:
from dask.distributed import Client

client = Client("tcp://10.0.1.207:41553")
client

<Client: 'tcp://10.0.1.207:41553' processes=2 threads=12, memory=96.00 GiB>

## GeoTIFF Export

Export the rasterized features from the target grid to GeoTIFF files.

In [7]:
# output handle: AHN4 dataset, mask for building, road and water, target grid spacing 10m
output_handle = 'ahn4_mask-building-road-water_10m'

# setup input dictionary to configure the geotiff export pipeline
# NOTE: to export the geotiffs we have mounted the dCache storage with rclone
geotiff_export_input_classification = {
    'setup_local_fs': {
        'input_folder': path_input,
        'output_folder': path_output
    },
    'parse_point_cloud': {},
    'data_split': {'xSub': 1, 'ySub': 1},
    'create_subregion_geotiffs': {'output_handle': output_handle},
    'pushremote': path_output.as_posix(),
#     'cleanlocalfs': {}  # DO NOT CLEAN - it would erase remote input..
}


# write input dictionary to JSON file
with open('geotiff_export_input_classification.json', 'w') as f:
    json.dump(geotiff_export_input_classification, f)

In [ ]:
macro = MacroPipeline()

gw = GeotiffWriter(bands='ground_type', label='ground_type').setup_webdav_client(wd_opts).config(geotiff_export_input_classification)
macro.add_task(gw)

macro.setup_cluster(cluster=cluster)

# run!
macro.run()

# save outcome results and write name of failed pipelines to file
macro.print_outcome(to_file='geotiff_export_classification.out')
failed = macro.get_failed_pipelines()
if failed:
    with open('geotiff_export_classification_failed.json', 'w') as f:
        json.dump([pip.label for pip in failed], f)
    raise RuntimeError('Some of the pipelines have failed')

In [ ]:
from dask.distributed import Client, Future
client = Client('tcp://145.100.59.123:8786')
futures = [Future(key) for key in client.who_has().keys()]
client.cancel(futures)

## Terminate cluster

In [ ]:
macro.shutdown()